In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

using Catalyst, DifferentialEquations, Plots, DataFrames

  Activating project at `~/Code/intro_julia_project`


In [2]:
function active_rtcr(rt, kr, L, c, rtcr)
    alpha = rt / kr
    fa = (1 + alpha)^6 / (L * ((1 + c * alpha)^6) + (1 + alpha)^6)
    return fa * rtcr
end

function tscr_ab(Vmax_init, atp, Km_init, rt, kr, L, c, rtcr, k_diss, ω_ab, θtscr)
    Voc = Vmax_init * atp / (Km_init + atp)  
    sig_o = active_rtcr(rt, kr, L, c, rtcr) * Voc / k_diss 
    return sig_o * ω_ab * atp / (θtscr + atp) 
end

tscr_r(ω_r, atp, θtscr) = ω_r * atp / (θtscr + atp)

function tlr(g_max, atp, θtlr, kc, rh, rm_x, nx)
    tlr_el = g_max*atp/(θtlr+atp)
    return (1 / nx) * kc * rh * rm_x * tlr_el
end

Vrep(rtcb, rt, krep, km_b) = rtcb * rt * krep / (rt + km_b) 
Vtag(rtca, rd, ktag, km_a) = rtca * rd * ktag / (rd + km_a) 


Vtag (generic function with 1 method)

In [6]:
rtc_model = @reaction_network begin

    tscr_ab(Vmax_init, atp, Km_init, rt, kr, L, c, rtcr, k_diss, ω_ab, θtscr),  ∅ --> (rm_a, rm_b)
    # tscr_ab(Vmax_init, atp, Km_init, rt, kr, L, c, rtcr, k_diss, ω_ab, θtscr),  ∅ --> rm_b
    tscr_r(ω_r, atp, θtscr),                                                    ∅ --> rm_r
    tlr(g_max, atp, θtlr, kc, rh, rm_a, na),                                  ∅ --> rtca #rm_a + rh --> rtca
    tlr(g_max, atp, θtlr, kc, rh, rm_b, nb),                                  ∅ --> rtcb #rm_b + rh --> rtcb
    tlr(g_max, atp, θtlr, kc, rh, rm_r, nr),                                  ∅ --> rtcr #rm_r + rh --> rtcr
    kin,                                             ∅ --> rh
    Vtag(rtca, rd, ktag, km_a),                                                 rd --> rt #rd + rtca --> rt
    Vrep(rtcb, rt, krep, km_b),                                                 rt --> rh #rt + rtcb --> rh
    kdam,                                                                          rh --> rd
    kdeg,                                                                          rd --> ∅
    d,                                                                             (rm_a, rm_b, rm_r) --> ∅

    lam,                                                                         (rm_a, rm_b, rm_r, rtca, rtcb, rtcr, rh, rd, rt) --> ∅

end;

In [7]:
odesys = convert(ODESystem, rtc_model)

Model ##ReactionSystem#247:
Equations (9):
  9 standard: see equations(##ReactionSystem#247)
Unknowns (9): see unknowns(##ReactionSystem#247)
  rm_a(t)
  rm_b(t)
  rm_r(t)
  rtca(t)
  ⋮
Parameters (25): see parameters(##ReactionSystem#247)
  Vmax_init
  atp
  Km_init
  kr
  ⋮